In [5]:
import json, os, sys, random, time, copy
from collections import Counter, defaultdict
import numpy as np
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
from pprint import pprint
import pickle

In [2]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]


In [3]:
def contentUrl_to_displayUrl(contentUrl):
    if 'commons' in contentUrl:
        prefix = 'https://commons.wikimedia.org/wiki/'
    else:
        prefix = 'https://en.wikipedia.org/wiki/'
    tokens = contentUrl.split('/')
    if 'px-' in tokens[-1]:
        return prefix + 'File:'+tokens[-2]
    else:
        return prefix + 'File:'+tokens[-1]

In [4]:
def displayUrl_to_imgUrl(html):

    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find_all('a',class_= "mw-thumbnail-link", limit=10)
    idx = 0
    default = ""
    for r in result:
        soup_short = BeautifulSoup(str(r), 'html.parser')
        for a in soup_short.find_all('a', href=True):
            href = a['href']
            if idx == 0: 
                default = a['href']
                idx += 1
            if '800px' in href:
                return href, "GOOD"
    if len(default)>0: return default, "GOOD"
    else:
        result = soup.find_all('div',class_= "fullImageLink", limit=1)
        for r in result:
            soup_short = BeautifulSoup(str(r), 'html.parser')
            for im in soup_short.find_all('img'):
                if max(int(im['width']), int(im['height']))>=600 and min(int(im['width']), int(im['height']))>=400:
                    return im['src'], "GOOD"
                return im['src'], "SIZE"
    return "", "FIELD_NONEXIST"

In [5]:
def get_display_page_html(desurl):
    start = time.time()
    try:
        req = urllib.request.Request(desurl, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
        with urllib.request.urlopen(req) as f:
            html = f.read().decode('utf-8')
        #print("url request time: ", time.time() - start)
        return html
    except KeyboardInterrupt:
        raise
    except:
        return ""

In [8]:
def normalize_imgUrl(url):
    if not 'commons' in url: return "https:"+url
    displayUrl = contentUrl_to_displayUrl(url)
    html = get_display_page_html(displayUrl)
    imgUrl, status = displayUrl_to_imgUrl(html)
    return imgUrl

In [17]:
data = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data/upd_txt_data_{}.json".format(1000), "r"))
try: data_upd_imgUrl = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "r"))
except: data_upd_imgUrl = {}
for k in range(50):
    if str(k) in data_upd_imgUrl: continue
    data_upd_imgUrl[str(k)] = copy.deepcopy(data[str(k)])
    data_upd_imgUrl[str(k)]['img_negFacts'] = []
    for f in data[str(k)]['img_negFacts']:
        if "maps.wikimedia.org" in f['imgUrl']: continue
        f_upd = copy.deepcopy(f)
        f_upd['imgUrl'] = normalize_imgUrl(f['imgUrl'])
        data_upd_imgUrl[str(k)]['img_negFacts'].append(f_upd)
    if k % 10 == 0:
        json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)
#json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [25]:
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_16k/"

snippets_count = []
img_count = []

no_snippet_k = []
no_img_k = []

files = os.listdir(path)
data = {}
for f in files:
    if not '.json' in f: continue
    x = json.load(open(os.path.join(path, f), "r"))
    for k in x:
        num_snippets = len(x[k]['new_negFacts'])
        num_imgs = len(x[k]['img_negFacts'])
        if num_snippets == 0:
            no_snippet_k.append(k)
        if num_imgs == 0:
            no_img_k.append(k)
        snippets_count.append(num_snippets)
        img_count.append(num_imgs)
    data.update(x)
print("{} samples finished".format(len(data)))
print(sorted(Counter(snippets_count).items()))
print(sorted(Counter(img_count).items()))
print(len([k for k in data if len(data[k]['new_negFacts'])< 5 or len(data[k]['img_negFacts']) < 5]))

17430 samples finished
[(0, 813), (1, 579), (2, 504), (3, 456), (4, 394), (5, 362), (6, 366), (7, 370), (8, 319), (9, 368), (10, 303), (11, 277), (12, 287), (13, 262), (14, 255), (15, 236), (16, 229), (17, 218), (18, 253), (19, 228), (20, 213), (21, 208), (22, 177), (23, 190), (24, 191), (25, 171), (26, 162), (27, 155), (28, 139), (29, 139), (30, 159), (31, 159), (32, 147), (33, 159), (34, 150), (35, 128), (36, 142), (37, 134), (38, 124), (39, 106), (40, 7198)]
[(0, 330), (1, 696), (2, 636), (3, 615), (4, 537), (5, 515), (6, 459), (7, 454), (8, 430), (9, 443), (10, 394), (11, 364), (12, 331), (13, 330), (14, 339), (15, 305), (16, 292), (17, 281), (18, 305), (19, 260), (20, 251), (21, 219), (22, 217), (23, 223), (24, 195), (25, 224), (26, 199), (27, 220), (28, 144), (29, 190), (30, 186), (31, 159), (32, 159), (33, 158), (34, 148), (35, 152), (36, 146), (37, 125), (38, 147), (39, 139), (40, 5513)]
4213


In [26]:
# 0725: imgUrl was updated so that we'll use the 800px version if it exists. And imgs from map.wikimedia.org were deleted
# 0803: distractor mining done for 16k new txt data 
print(len(data))
#json.dump(data, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0725.json", "w"), indent=4)
json.dump(data, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0803_16k.json", "w"), indent=4)

17430


In [27]:
Counter([data[k]['Split'] for k in data])

Counter({'ind_test': 1805, 'ood_test': 1500, 'train': 12314, 'val': 1811})

In [9]:
src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/dis_src2idx-Copy1.pkl", "rb"))
print(len(src2idx))

340194


In [10]:
for s in src2idx:
    print(type(src2idx[s]))
    print(s)
    print(src2idx[s])
    break

<class 'int'>
https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Central-Park-18.jpg/800px-Central-Park-18.jpg
10000000


In [28]:
# Assign img ids for neg_img for txt data (starting from 20000000)
#txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0725_random_split.json", "r"))

# Assign img ids for neg_img for 16k txt data (starting from 20098051)
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0803_16k.json", "r"))
print(len(txt_dataset))

17430


In [30]:
x_dis_src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "rb"))
print(max(x_dis_src2idx.values()))

20098050


In [31]:
#x_dis_src2idx = {}
#idx = 20000000
x_dis_src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "rb"))
print(max(x_dis_src2idx.values()))
idx = max(x_dis_src2idx.values()) + 1
for k in txt_dataset:
    for im in txt_dataset[k]['img_negFacts']:
        s = im['imgUrl']
        if s in x_dis_src2idx: continue
        x_dis_src2idx[s] = idx
        idx += 1
print(idx)
print(len(x_dis_src2idx))

20098050
20240700
240700


In [32]:
x_dis_idx2src = {}
for s in x_dis_src2idx:
    idx = x_dis_src2idx[s]
    x_dis_idx2src[idx] = s
print(len(x_dis_idx2src))

240700


In [33]:
# Jul 27 src2idx fixed for negImgs in 7921 txt data
# Aug 3 src2idx fixed for negImgs for 7921 + 16k data
pickle.dump(x_dis_src2idx, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "wb"))

In [34]:
# Jul 27 idx2src fixed for negImgs in 7921 txt data
# Aug 3 idx2src fixed for negImgs for 7921 + 16k data
pickle.dump(x_dis_idx2src, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_idx2src.pkl", "wb"))

In [63]:
print(len(os.listdir("/data/yingshac/MMMHQA/x_distractors")))

235954


In [21]:
x_dis_idx2src[20070025]

'https://upload.wikimedia.org/wikipedia/commons/f/fd/Adam_and_the_Angels_watched_by_Iblis.jpg'